Could I make an autofocus algorithm based on finding the size of something using ML then using the focal length of my camera to find it's distance, then jumping to that distance and CBAFing from there?

setup something like iphone on a rig with a camera that has an external cinema AF motor, then go as fast as possible lol. Main problem would be the inference time of course, but cool little side POC for when cameras have nvidia GPUs in 10 years? lol

If I really get into it, maybe mount the camera to a turret lmao


Using an existing model on HF for species classification(I've made one in the past, but this seems to be a lot better lol)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!git clone https://github.com/AlexeyAB/darknet.git

# Change directory to darknet
%cd darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15851, done.
remote: Total 15851 (delta 0), reused 0 (delta 0), pack-reused 15851
Receiving objects: 100% (15851/15851), 14.37 MiB | 17.69 MiB/s, done.
Resolving deltas: 100% (10683/10683), done.
/content/darknet


In [7]:
#Edit the Makefile file, change GPU, CUDNN, and OPENCV to 1 then run
!make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘bu

In [8]:
# Download pre-trained YOLOv3 weights
!wget https://pjreddie.com/media/files/yolov3.weights


--2024-06-29 17:29:35--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  35.0MB/s    in 7.4s    

2024-06-29 17:29:43 (31.8 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [9]:
# Importing the libraries needed
import torch
import urllib.request
from PIL import Image
from transformers import pipeline

# Determining the file URL
url = 'some url'

# Opening the image using PIL
img = Image.open(urllib.request.urlretrieve(url)[0])

# Loading the model and preprocessor using Pipeline
pipe = pipeline("image-classification", model="dennisjooo/Birds-Classifier-EfficientNetB2")

# Running the inference
result = pipe(img)[0]

# Printing the result label
print(result['label'])


ValueError: unknown url type: 'some url'

In [35]:
import cv2
import numpy as np
%cd
%cd /content
# Load image
image_path = 'cardinal.jpg'
image = cv2.imread(image_path)

# Check if image loaded successfully
if image is None:
    print(f"Error: Could not load image from {image_path}. Check if the file exists and you have read permissions.")
else:
    height, width = image.shape[:2]

    # Convert to blob for YOLO input
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    # Proceed with further processing


/root
/content


In [37]:
# Set up YOLOv3
net = cv2.dnn.readNetFromDarknet('darknet/yolov7-tiny.cfg', 'darknet/yolov7-tiny.weights')
layer_names = net.getLayerNames()
output_layers_indices = net.getUnconnectedOutLayers()

# Handle the case where output_layers_indices is a scalar or an array
if isinstance(output_layers_indices, np.ndarray):
    output_layers = [layer_names[i - 1] for i in output_layers_indices] # Iterate over elements directly
else:
    output_layers = [layer_names[output_layers_indices - 1]] # Handle the scalar case

# Forward pass through network
net.setInput(blob)
outs = net.forward(output_layers)

In [45]:
class_ids = []
confidences = []
boxes = []

# Process detections
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5 and class_id == 14:  # Assuming class_id 14 is for bird (adjust if necessary)
            # Calculate bounding box coordinates
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Non-maximum suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

# Calculate diagonal lengths
image_diagonal = np.sqrt(width**2 + height**2)

if len(indices) > 0:
    for i in indices.flatten():
        x, y, w, h = boxes[i]
        bird_diagonal = np.sqrt(w**2 + h**2)
        bird_percent = (bird_diagonal / image_diagonal) * 100
        print(f'Bird occupies {bird_percent:.2f}% of the image diagonal length.')



Bird occupies 74.74% of the image diagonal length.


In [54]:
def calculate_distance(bird_percent, bird_size_cm, focal_length_mm):
    # Convert bird size from centimeters to millimeters
    bird_size_mm = bird_size_cm * 10.0  # 1 cm = 10 mm

    # Calculate the real size of the bird in millimeters
    bird_real_size_mm = bird_size_mm / bird_percent * 100  # Real size in mm

    # Calculate distance using the formula: distance = (real_size_mm * focal_length_mm) / apparent_size_mm
    distance_mm = (bird_real_size_mm * focal_length_mm) / bird_size_mm

    # Convert distance from millimeters to meters (1 mm = 0.001 meters)
    distance_meters = distance_mm * 0.001

    return distance_meters



In [55]:
calculate_distance(bird_percent,21,85)

0.85

In [50]:
# Example usage
bird_percent = 10.0  # Example: Bird occupies 10% of the image
bird_size_cm = 20.0  # Example: Size of the bird in centimeters (e.g., width or height of the bounding box)
focal_length_mm = 500 # Example: Focal length of the camera in millimeters

distance_meters = calculate_distance(bird_percent, bird_size_cm, focal_length_mm)
print(f"The bird is approximately {distance_meters:.2f} meters away from the camera.")



The bird is approximately 16.40 meters away from the camera.
